In [2]:
import pandas as pd
import os
import torch
import numpy as np
import matplotlib
import sys; sys.path.append("..")
import utils.constant as constant

from tqdm.notebook import tqdm, tqdm_notebook
from datetime import datetime, timedelta

from IPython.display import clear_output

path_dataset = constant.PATH_MIMIC_III

list_csvgz = [f for f in os.listdir(path_dataset) if f[-6:]=="csv.gz"]
list_csvgz

['DRGCODES.csv.gz',
 'LABEVENTS.csv.gz',
 'PATIENTS.csv.gz',
 'D_ITEMS.csv.gz',
 'DIAGNOSES_ICD.csv.gz',
 'CAREGIVERS.csv.gz',
 'SERVICES.csv.gz',
 'CPTEVENTS.csv.gz',
 'PRESCRIPTIONS_PREPROCESSED.csv.gz',
 'OUTPUTEVENTS.csv.gz',
 'D_LABITEMS.csv.gz',
 'D_ICD_DIAGNOSES.csv.gz',
 'CALLOUT.csv.gz',
 'ADMISSIONS.csv.gz',
 'D_CPT.csv.gz',
 'INPUTEVENTS_MV.csv.gz',
 'D_LABITEMS_NEW.csv.gz',
 'PROCEDUREEVENTS_MV.csv.gz',
 'ADMISSIONS_NEW.csv.gz',
 'PRESCRIPTIONS.csv.gz',
 'NOTEEVENTS.csv.gz',
 'DATETIMEEVENTS.csv.gz',
 'MICROBIOLOGYEVENTS.csv.gz',
 'ICUSTAYS.csv.gz',
 'DRUGS_NDC_FEAT.csv.gz',
 'D_ICD_PROCEDURES.csv.gz',
 'CHARTEVENTS.csv.gz',
 'TRANSFERS.csv.gz',
 'PROCEDURES_ICD.csv.gz',
 'LABEVENTS_PREPROCESSED.csv.gz',
 'INPUTEVENTS_CV.csv.gz']

In [3]:
df_microbio = pd.read_csv(os.path.join(path_dataset, "MICROBIOLOGYEVENTS.csv.gz"))
df_microbio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631726 entries, 0 to 631725
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ROW_ID               631726 non-null  int64  
 1   SUBJECT_ID           631726 non-null  int64  
 2   HADM_ID              631726 non-null  int64  
 3   CHARTDATE            631726 non-null  object 
 4   CHARTTIME            589954 non-null  object 
 5   SPEC_ITEMID          631647 non-null  float64
 6   SPEC_TYPE_DESC       631726 non-null  object 
 7   ORG_ITEMID           326881 non-null  float64
 8   ORG_NAME             328016 non-null  object 
 9   ISOLATE_NUM          328016 non-null  float64
 10  AB_ITEMID            275834 non-null  float64
 11  AB_NAME              275834 non-null  object 
 12  DILUTION_TEXT        267350 non-null  object 
 13  DILUTION_COMPARISON  267178 non-null  object 
 14  DILUTION_VALUE       267178 non-null  float64
 15  INTERPRETATION   

In [4]:
df_microbio.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_microbio.ORG_NAME.value_counts()

STAPH AUREUS COAG +                   63947
ESCHERICHIA COLI                      60133
STAPHYLOCOCCUS, COAGULASE NEGATIVE    32777
KLEBSIELLA PNEUMONIAE                 30628
PSEUDOMONAS AERUGINOSA                28926
                                      ...  
CITROBACTER  KOSERI (DIVERSUS)            1
BACTEROIDES UREOLYTICUS GROUP             1
STEMPHYLIUM SP.                           1
NEISSERIA GONORRHOEAE                     1
ACREMONIUM SPECIES                        1
Name: ORG_NAME, Length: 362, dtype: int64

In [6]:
df_microbio.AB_NAME.value_counts()

GENTAMICIN              25770
LEVOFLOXACIN            18652
CEFTAZIDIME             14640
TOBRAMYCIN              14504
TRIMETHOPRIM/SULFA      14105
CEFEPIME                13465
MEROPENEM               12982
VANCOMYCIN              12915
OXACILLIN               12201
CIPROFLOXACIN           12023
PIPERACILLIN/TAZO       11549
ERYTHROMYCIN            11060
CEFTRIAXONE             10865
AMPICILLIN               9547
TETRACYCLINE             9165
PENICILLIN               9156
CLINDAMYCIN              8760
AMPICILLIN/SULBACTAM     8150
PIPERACILLIN             7836
CEFAZOLIN                7812
RIFAMPIN                 7130
NITROFURANTOIN           6428
IMIPENEM                 5990
CEFUROXIME               5527
PENICILLIN G             1905
LINEZOLID                1425
AMIKACIN                  946
CHLORAMPHENICOL           878
DAPTOMYCIN                445
CEFPODOXIME                 3
Name: AB_NAME, dtype: int64

In [8]:
mask = df_microbio.AB_ITEMID.notnull()
df_microbio.loc[mask].head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
27,771,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90012.0,GENTAMICIN,<=0.5,<=,1.0,S
28,772,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90016.0,OXACILLIN,0.5,=,1.0,S
29,773,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90006.0,ERYTHROMYCIN,=>8,=>,8.0,R
30,774,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90002.0,PENICILLIN,=>0.5,=>,1.0,R
31,775,106,145167,2192-08-09 00:00:00,2192-08-09 21:11:00,70062.0,SPUTUM,80023.0,STAPH AUREUS COAG +,1.0,90025.0,LEVOFLOXACIN,0.25,=,0.0,S
